# Extraindo tabela dos pdfs de Materiais de uso geral e informática

### Esse notebook tem o objetivo de:
    - [ ] Extrair a tabela nos pdfs
    - [ ] Padronizar o formato
    - [ ] Fatorar uma função que automatiza isso

### Prerequisitos:
    - [X] Baixar pelo menos um PDF para referencia

In [1]:
PDF = '../data/pdfs/materiais-de-uso-geral-e-informatica/materiaisdiversos.pdf'

In [22]:
import sys
sys.path.insert(0, '..')
from IPython.display import HTML

def show_pdf(path, page=0, height=400):
    return HTML(f'''
        <iframe
            src="{path}#page={page}"
            width=100%
            height={height}
            >
        </iframe>
    ''')

In [3]:
show_pdf(PDF)

In [4]:
from lib.materiais_de_uso_geral_e_informatica import (
    extract_table,
    format_table,
    extrair_df,
)
from tabula import read_pdf

In [5]:
pandas_options = {
    'dtype': str,
}

df = read_pdf(
    PDF,
    spreadsheet=True,
    pandas_options=pandas_options,
    pages='all',
)

df.head()

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO...",UN,"0,48"
1,2,7930.00.026-66,"ÁGUA SANITÁRIA, LÍQUIDO HOMOGÊNEO, GERMICIDA, ...",UN,"1,63"
2,3,6810.10.500-37,"ALCOOL, GEL FRASCO (REFIL) C/ MIN. 800ML M...",UN,"12,10"
3,4,6810.10.395-72,"ÁLCOOL ETÍLICO, A 70o, DE USO HOSPITALAR PARA ...",UN,"4,28"
4,5,6810.10.118-05,"ÁLCOOL ETÍLICO ANIDRO (ABSOLUTO), FILTRADO, 99...",UN,"6,31"


In [6]:
no = df.columns[0]
df[df[no] == no]

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
13,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
25,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
39,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
54,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
68,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
81,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
99,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
121,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
136,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
147,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço


In [7]:
df[df['CÓDIGO'].isnull()].index

Int64Index([152, 153, 154, 155], dtype='int64')

In [8]:
from typing import Iterable, Iterator, Tuple, Any
from itertools import takewhile, count, tee

def pairwise(it : Iterable[Any]) -> Iterator[Tuple[Any, Any]]:
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(it)
    next(b, None)
    return zip(a, b)

In [9]:
def take_consecutive(it):
    it = iter(it)
    it = pairwise(it)
    it = takewhile(lambda t: t[0] + 1 == t[1], it)
    it = list(it)
    it = [a for a, b in it] + [it[-1][1]]
    return it

In [10]:
take_consecutive(df[df['CÓDIGO'].isnull()].index)

[152, 153, 154, 155]

In [45]:
import pandas as pd
from lib.typing import Path
from lib.util import drop_repeated_headers

def extract_table(path: Path) -> pd.DataFrame:
    pandas_options = {
        'dtype': str,
    }
    
    return read_pdf(
        path,
        spreadsheet=True,
        pandas_options=pandas_options,
        pages='all',
    )


def format_table(df: pd.DataFrame,
                ) -> Tuple[pd.DataFrame,
                           pd.DataFrame]:
    df = drop_repeated_headers(df)
    start, *_, stop = take_consecutive(df[df['CÓDIGO'].isnull()].index)
    return df.iloc[:start], df.iloc[stop+1:].reset_index(drop=True)

In [12]:
df = extract_table(PDF)
df.head()

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO...",UN,"0,48"
1,2,7930.00.026-66,"ÁGUA SANITÁRIA, LÍQUIDO HOMOGÊNEO, GERMICIDA, ...",UN,"1,63"
2,3,6810.10.500-37,"ALCOOL, GEL FRASCO (REFIL) C/ MIN. 800ML M...",UN,"12,10"
3,4,6810.10.395-72,"ÁLCOOL ETÍLICO, A 70o, DE USO HOSPITALAR PARA ...",UN,"4,28"
4,5,6810.10.118-05,"ÁLCOOL ETÍLICO ANIDRO (ABSOLUTO), FILTRADO, 99...",UN,"6,31"


In [13]:
a, b = format_table(df)

In [14]:
a.head()

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO...",UN,"0,48"
1,2,7930.00.026-66,"ÁGUA SANITÁRIA, LÍQUIDO HOMOGÊNEO, GERMICIDA, ...",UN,"1,63"
2,3,6810.10.500-37,"ALCOOL, GEL FRASCO (REFIL) C/ MIN. 800ML M...",UN,"12,10"
3,4,6810.10.395-72,"ÁLCOOL ETÍLICO, A 70o, DE USO HOSPITALAR PARA ...",UN,"4,28"
4,5,6810.10.118-05,"ÁLCOOL ETÍLICO ANIDRO (ABSOLUTO), FILTRADO, 99...",UN,"6,31"


In [15]:
b.head()

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,7445.02.116-85,"CARTUCHO, TINTA, COLORIDA, PARA IMPRESSORA A J...",UN,"190,00"
1,2,7445.02.072-20,"CARTUCHO, TINTA COLORIDA, PARA IMPRESSORA A JA...",UN,"163,08"
2,3,7445.02.010-27,"CARTUCHO, TINTA COLORIDA, PARA IMPRESSORA A JA...",UN,"99,00"
3,4,7445.02.112-51,"CARTUCHO, TINTA COLORIDA, PARA IMPRESSORA A JA...",UN,"157,50"
4,5,7445.02.080-30,"CARTUCHO, TINTA, COLORIDA, PARA IMPRESSORA A J...",UN,"224,10"


### Tem um caso especial que precisa ser divido de forma dirente que os outros

In [24]:
SPECIAL_CASE = '../data/pdfs/materiais-de-uso-geral-e-informatica/catalogo_materiais_diversos.pdf'

In [25]:
show_pdf(SPECIAL_CASE, page=11, height=600)

In [26]:
df = extract_table(SPECIAL_CASE)
df.head()

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO...",UN,"0,48"
1,2,7930.00.026-66,"ÁGUA SANITÁRIA, LÍQUIDO HOMOGÊNEO, GERMICIDA, ...",UN,"1,56"
2,3,6810.10.500-37,"ALCOOL, GEL FRASCO (REFIL) C/ MIN. 800ML M...",UN,"8,30"
3,4,6810.10.395-72,"ÁLCOOL ETÍLICO, A 70o, DE USO HOSPITALAR PARA ...",UN,"6,25"
4,5,6810.10.118-05,"ÁLCOOL ETÍLICO ANIDRO (ABSOLUTO), FILTRADO, 99...",UN,"5,99"


In [27]:
df = drop_repeated_headers(df)
df.head()

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO...",UN,"0,48"
1,2,7930.00.026-66,"ÁGUA SANITÁRIA, LÍQUIDO HOMOGÊNEO, GERMICIDA, ...",UN,"1,56"
2,3,6810.10.500-37,"ALCOOL, GEL FRASCO (REFIL) C/ MIN. 800ML M...",UN,"8,30"
3,4,6810.10.395-72,"ÁLCOOL ETÍLICO, A 70o, DE USO HOSPITALAR PARA ...",UN,"6,25"
4,5,6810.10.118-05,"ÁLCOOL ETÍLICO ANIDRO (ABSOLUTO), FILTRADO, 99...",UN,"5,99"


In [37]:
df[df[df.columns[0]] == '1'].index

Int64Index([0, 144], dtype='int64')

### Mais um caso especial:

In [41]:
SPECIAL_CASE = '../data/pdfs/materiais-de-uso-geral-e-informatica/materiais_diversos_4_2016.pdf'

In [43]:
show_pdf(SPECIAL_CASE)

In [42]:
df = extract_table(SPECIAL_CASE)
df.head()

,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO, 40X10X1CM, PARA USO PÓS-PARTO CONFECCIONADO COM MATERIAL ABSORVENTE, HIPOALERGÊNICO, COM GEL DISTRIBUÍDO DE FORMA A GARANTIR UMA BOA ABSORÇÃO DE FLUXO, COM COBERTURA INTERNA DE FALSO TECIDO, REVESTIDO EXTERNAMENTE POR PELÍCULA IMPERMEÁVEL, DIMENSÕES MÍNIMAS DA ÁREA ABSORVÍVEL 40XABSORVENTE HIGIÊNICO PÓS- PARTO, HIPOALERGÊNICO, 40X10X1CM, CONFECCIONADO COM MATERIAL ABSORVENTE 40X10X1CM, EMBALAGEM CONTENDO EXTERNAMENTE DADOS DE IDENTIFICAÇÃO E PROCEDÊNCIA, DATA DA FABRICAÇÃO E TEMPO DE VALIDADE",UN,"0,48"
0,2,7930.00.026-66,"ÁGUA SANITÁRIA, LÍQUIDO HOMOGÊNEO, GERMICIDA, ...",UN,"1,56"
1,3,6810.10.500-37,"ALCOOL, GEL FRASCO (REFIL) C/ MIN. 800ML M...",UN,"8,30"
2,4,6810.10.395-72,"ÁLCOOL ETÍLICO, A 70o, DE USO HOSPITALAR PARA ...",UN,"5,99"
3,5,6810.10.118-05,"ÁLCOOL ETÍLICO ANIDRO (ABSOLUTO), FILTRADO, 99...",UN,"5,99"
4,6,6810.10.502-07,"ALCOOL ETILICO HIDRATADO, GEL, 65G. A 70G.INPM...",UN,"6,80"


In [48]:
df[df[df.columns[0]] == '1']

,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO, 40X10X1CM, PARA USO PÓS-PARTO CONFECCIONADO COM MATERIAL ABSORVENTE, HIPOALERGÊNICO, COM GEL DISTRIBUÍDO DE FORMA A GARANTIR UMA BOA ABSORÇÃO DE FLUXO, COM COBERTURA INTERNA DE FALSO TECIDO, REVESTIDO EXTERNAMENTE POR PELÍCULA IMPERMEÁVEL, DIMENSÕES MÍNIMAS DA ÁREA ABSORVÍVEL 40XABSORVENTE HIGIÊNICO PÓS- PARTO, HIPOALERGÊNICO, 40X10X1CM, CONFECCIONADO COM MATERIAL ABSORVENTE 40X10X1CM, EMBALAGEM CONTENDO EXTERNAMENTE DADOS DE IDENTIFICAÇÃO E PROCEDÊNCIA, DATA DA FABRICAÇÃO E TEMPO DE VALIDADE",UN,"0,48"
143,1,7445.02.116-85,"CARTUCHO, TINTA, COLORIDA, PARA IMPRESSORA A J...",UN,"155,57"


In [56]:
def extract_table(path: Path) -> pd.DataFrame:
    pandas_options = {
        'dtype': str,
        'header': None,
        'names': ['N°',  'CÓDIGO', 'Materiais Diversos', 'Unid', 'R$ Preço'],
    }
    
    return read_pdf(
        path,
        spreadsheet=True,
        pandas_options=pandas_options,
        pages='all',
    )

In [63]:
df = extract_table(SPECIAL_CASE)
df.head()

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO...",UN,"0,48"
1,2,7930.00.026-66,"ÁGUA SANITÁRIA, LÍQUIDO HOMOGÊNEO, GERMICIDA, ...",UN,"1,56"
2,3,6810.10.500-37,"ALCOOL, GEL FRASCO (REFIL) C/ MIN. 800ML M...",UN,"8,30"
3,4,6810.10.395-72,"ÁLCOOL ETÍLICO, A 70o, DE USO HOSPITALAR PARA ...",UN,"5,99"
4,5,6810.10.118-05,"ÁLCOOL ETÍLICO ANIDRO (ABSOLUTO), FILTRADO, 99...",UN,"5,99"


In [64]:
df[df[df.columns[0]] == '1']

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO...",UN,"0,48"
144,1,7445.02.116-85,"CARTUCHO, TINTA, COLORIDA, PARA IMPRESSORA A J...",UN,"155,57"


In [66]:
df = extract_table(PDF)
df = drop_repeated_headers(df)
df.head()

,N°,CÓDIGO,Materiais Diversos,Unid,R$ Preço
0,1,8540.00.019-76,"ABSORVENTE HIGIÊNICO PÓS-PARTO, HIPOALERGÊNICO...",UN,"0,48"
1,2,7930.00.026-66,"ÁGUA SANITÁRIA, LÍQUIDO HOMOGÊNEO, GERMICIDA, ...",UN,"1,63"
2,3,6810.10.500-37,"ALCOOL, GEL FRASCO (REFIL) C/ MIN. 800ML M...",UN,"12,10"
3,4,6810.10.395-72,"ÁLCOOL ETÍLICO, A 70o, DE USO HOSPITALAR PARA ...",UN,"4,28"
4,5,6810.10.118-05,"ÁLCOOL ETÍLICO ANIDRO (ABSOLUTO), FILTRADO, 99...",UN,"6,31"
